In [1]:
import pandas as pd
import numpy as np
import os
import string
from datetime import tzinfo, timedelta, datetime

os.getcwd()
import glob



In [2]:
def convert_mm(value):
    #length in mm transformed to float
    try:
        new_value = value.replace(' mm','')
        return float(new_value)    
    except AttributeError:
        return np.NaN

In [3]:
def convert_exposure(value):
    #length in mm transformed to float
    try:
        list_string=value.split('/')
        if (len(list_string)==1):
            return float(value)
        else:
            return float(list_string[0])/float(list_string[1])
    except AttributeError:
        #print ('WTF',value)
        return np.NaN
    #return float(0)
        

In [4]:
def remove_full_path(value):
    #all pictures originate from a Pictures folder, remove the full path
    #including Pictures
    try:
        list_string=value.split('/Pictures')
        return list_string[1]
    except AttributeError:
        return np.NaN

In [5]:
def funcDayPeriod(filename):
    search_night=["bei Nacht",'Sternenhimmel',"Mondschein","Mond","Ice and Snow/"]
    search_night_special=["Lumiere","euerwerk","Firework","Freudenfeuer"]
    search_twilight=["bei anbrechender Nacht","Abenddaemmerung","Morgendaemmerung","Sonnenaufgang"]
    if any (x in filename for x in search_night):
        return -1
    elif any(x in filename for x in search_night_special):
         return 0
    elif any(x in filename for x in search_twilight):
        return 3
    elif "bei Sonnenuntergang" in filename:
        return 2
    else:
        return 1

In [6]:
def funcComposite(filename):
    if "Panorama." in filename:
        return 1
    else:
        return 0

In [7]:
def funcMovie(filename):
    search_file_names=[".JPG",".jpg",".JPEG",".jpeg"]
    if any(x in filename for x in search_file_names):
        return 0
    else:
        return 1

In [8]:
def funcTimeZoneCorrectionNYCBias(value,filename,modelname,data_min,data_max):
    try:
        if value=='0000:00:00 00:00:00' or value=='missing':
            return np.NaN
        else:
            #date example: Dec 09 Sat ,2006, 17:12:27
            #abrMonth day(number zero-padded) day(abbr name),year, hour(24):minute:second
            #date is ordered by 
            date_orig = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S") 
            date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S") 
            datemin = datetime.strptime(data_min,"%b %d %a ,%Y, %H:%M:%S") 
            datemax = datetime.strptime(data_max,"%b %d %a ,%Y, %H:%M:%S") 
            if modelname=="SP500UZ":
                if(date>datemin and date<datemax):
                #never changed timezone after getting back from NYC, all messed up
                    date=date+timedelta(hours=6)
                    #print('date before',date_orig,'date after',date)
            return date.strftime("%b %d %a ,%Y, %H:%M:%S")
    except AttributeError:
        return np.NaN

In [61]:
def funcTimeZoneCorrection(value,filename,modelname):
    try:
        if value=='0000:00:00 00:00:00' or value=='missing':
            return np.NaN
        else:
            #date example: Dec 09 Sat ,2006, 17:12:27
            #abrMonth day(number zero-padded) day(abbr name),year, hour(24):minute:second
            date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S") 
            search_names_NYC_time_zone=["- Philadelphia,","New York City,","Newark,","Washington Airport"]
            search_names_London_time_zone_CANON600D=["Dublin & Nordirland/P","120830-120909 - Portugal/","Suedengland/","141212-141214 - London/","Greater London"]
            search_names_Istanbul_time_zone_CANON600D=["41023-141027 - Istanbul/","150828-150912 Sankt Petersburg/"]
            search_names_pacific_time_zone_CANON600D=["150613 - Los Angeles/","USA/P818"," - Grand Canyon Village, "," - Boulder City, "," - Las Vegas, "]
            search_names_China_time_zone_CANON80D=["180101 China/","/171230 Shanghai"," - China, "]
            search_names_EastEurope_time_zone_CANON80D=["170603 Athen/","171006 Tallinn ","/170414 Kiev"," - Tuerkei, "," - Irak, "," - Kuwait"," - Qatar, "]
            search_names_London_time_zone_CANON80D=[" - Saudi-Arabien,","/190413-190417 Suedostengland/"," - United Kingdom - ","/171110 Marrakesch/","170915 London/P","/180602 UK/"]
            search_names_Iceland_time_zone_CANON80D=[" - Island, ","/180328 Island/"]
            search_names_Gulf_time_zone_CANON80D=[" - Island, "," - Dubai, "]
            search_names_Pakistan_time_zone_CANON80D=[" - Pakistan, "]
            search_names_Korea_time_zone_CANON80D=[" - Paju, "," - Suwon, "," - Seoul, "," - Busan,"," - Incheon, "," - Gyeongju, "," - Gwanju, "]
            if modelname=="SP500UZ":
                #no change of default time, thus London time is an hour too late
                if "London/P" in filename:
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    date=date-timedelta(hours=1)
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S")
                elif any(x in filename for x in search_names_NYC_time_zone):
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    date=date-timedelta(hours=6)
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S")
                elif " Chicago - Edit2012/P" in filename:
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    date=date-timedelta(hours=7)
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S")      
                else:
                    return value;
            if modelname=="Canon EOS 600D":
                #all of EOS600D done including 2015
                if any(x in filename for x in search_names_London_time_zone_CANON600D):
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    date=date-timedelta(hours=1)
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S")
                elif any(x in filename for x in search_names_Istanbul_time_zone_CANON600D):
                    date=date+timedelta(hours=1)
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S")
                elif any(x in filename for x in search_names_pacific_time_zone_CANON600D):
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    date=date-timedelta(hours=9)
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S")
                elif "- Chicago, " in filename:
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    date=date-timedelta(hours=7)
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S")
                elif " - Washington D.C., " in filename:
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    date=date-timedelta(hours=6)
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S")
                elif "/160813 - USA/" in filename:
                    #remaining 2016 US trip times
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    date=date-timedelta(hours=8)
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S")
                elif "161205 - Japan/" in filename:
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    date=date+timedelta(hours=7)
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S")
                else:
                    #change to default summer time
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S")
            if modelname=="Canon PowerShot S110":  
                 #all running on summer time
                if any(x in filename for x in search_names_London_time_zone_CANON600D):
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    date=date-timedelta(hours=2)
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S")
                elif "41023-141027 - Istanbul/" in filename:
                    #for Istanbul timezone was not aligned with CANON EOS600D
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    date=date-timedelta(hours=1)
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S")
                elif "150613 - Los Angeles/" in filename:
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    date=date-timedelta(hours=10)
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S")
                elif "/2016/" in filename:
                    if "161205 - Japan/" in filename:
                        date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                        date=date+timedelta(hours=6)
                        return date.strftime("%b %d %a ,%Y, %H:%M:%S")
                    else:
                        #change to default summer time
                        date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                        date=date-timedelta(hours=1)
                        return date.strftime("%b %d %a ,%Y, %H:%M:%S")
                else:
                    #change to default summer time
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    date=date-timedelta(hours=1)
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S")
            if modelname=="Canon EOS 80D":
                if any(x in filename for x in search_names_China_time_zone_CANON80D):
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    date=date+timedelta(hours=6)
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S")
                elif any(x in filename for x in search_names_EastEurope_time_zone_CANON80D):
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    date=date+timedelta(hours=1)
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S")
                elif any(x in filename for x in search_names_London_time_zone_CANON80D):
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    date=date-timedelta(hours=1)
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S")
                elif " - Groenland" in filename:
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    date=date-timedelta(hours=4)
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S") 
                elif any (x in filename for x in search_names_Iceland_time_zone_CANON80D):
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    date=date-timedelta(hours=2)
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S") 
                elif "/170609 USA/" in filename:
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    date=date-timedelta(hours=8)
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S")  
                elif any(x in filename for x in search_names_Gulf_time_zone_CANON80D):
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    date=date+timedelta(hours=2)
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S")
                elif any(x in filename for x in search_names_Pakistan_time_zone_CANON80D):
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    date=date+timedelta(hours=3)
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S")
                elif any(x in filename for x in search_names_Korea_time_zone_CANON80D):
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    date=date+timedelta(hours=7)
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S")
                else:
                    #always on summer time
                    date = datetime.strptime(value,"%b %d %a ,%Y, %H:%M:%S")  
                    return date.strftime("%b %d %a ,%Y, %H:%M:%S")
    except AttributeError:
        return np.NaN

In [62]:
dataframe_list=[]

data_max=datetime.now()
datamax=data_max.strftime("%b %d %a ,%Y, %H:%M:%S")
data_min=datetime.now()
datamin=data_min.strftime("%b %d %a ,%Y, %H:%M:%S")

#print(datamin,datamax)

#push-back day related periods, 1 for day,2 for sunset,3 for dusk and daw,sunrise, 0 for night
for input in range(2006,2020):
    #print("get to year",input)
    df=pd.read_csv("outputEXIFcsvs/PhotoEXIFDataDayType_%s.csv"%(str(input)))
    #push back year index as additional list 
    #print(df.columns,df.head)
    #print(df.dtypes)
    df['SourceFile']=df['SourceFile'].apply(remove_full_path)    
    df['FocalLength']=df['FocalLength'].apply(convert_mm) 
    df['ExposureTime']=df['ExposureTime'].apply(convert_exposure)
    df['DateTimeOriginal']=df['DateTimeOriginal'].fillna('missing')
    df['year']=int(input)
    df['dayperiod']=df.apply(lambda x: funcDayPeriod(x['SourceFile']), axis=1)
    df['CompositePhoto']=df.apply(lambda x: funcComposite(x['SourceFile']), axis=1)
    df['Movie']=df.apply(lambda x: funcMovie(x['SourceFile']), axis=1)
    df.rename(columns={'DateTimeOriginal':'Date','FocalLength':'FocalLength in mm'},inplace=True) 
    df['Date']=df.apply(lambda x: funcTimeZoneCorrection(x['Date'],x['SourceFile'],x['Model']), axis=1)
    df['Date']=df['Date'].fillna('missing')
    total_rows=df['SourceFile'].count()
    if(input==2009 or input ==2010):
        if input==2009:
            test=df.loc[df['SourceFile'].str.contains("P7080022 - New York City")]
            #print("correction should start somewhere here",test,"index is",test.index.values.astype(int)[0])
            datamin=df.at[test.index.values.astype(int)[0],"Date"]   
            df['Date']=df.apply(lambda y: funcTimeZoneCorrectionNYCBias(y['Date'],y['SourceFile'],y['Model'],datamin,datamax), axis=1)
        elif input==2010:    
            test=df.loc[df['SourceFile'].str.contains("P8270068 - Le Locle, unterirdische Muehlen")]
            #print("correction should end somewhere here",test,"index is",test.index.values.astype(int)[0])
            datamax=df.at[test.index.values.astype(int)[0],"Date"]   
            df['Date']=df.apply(lambda y: funcTimeZoneCorrectionNYCBias(y['Date'],y['SourceFile'],y['Model'],datamin,datamax), axis=1)
    dataframe_list.append(df)
    #print("rows per year", input,total_rows)
    #print(df.dtypes)
 
combine_all_data=pd.DataFrame(dataframe_list[0]) 
#print(combine_all_data.head)
#print("now for the rest")
dataframe_list.pop(0)
for data in dataframe_list:
    #print(data.head)
    combine_all_data=pd.concat([combine_all_data,data],ignore_index=True,sort=False)
    #print(combine_all_data.head)
    
combine_all_data.to_csv(r'EXIFDataCombinedCameras_2006_to_2019_DateLocalTime.csv')
print(combine_all_data.head)

<bound method NDFrame.head of                                               SourceFile  \
0      /2006/060710 - London/P7100001 - Basel, Easyje...   
1      /2006/060710 - London/P7100002 - London, Leice...   
2      /2006/060710 - London/P7100003 - London, House...   
3      /2006/060710 - London/P7100004 - London, Westm...   
4      /2006/060710 - London/P7100005 - London, St. M...   
...                                                  ...   
91541  /2019/190715 Bings Birthday/P7140010 - Genf, R...   
91542  /2019/190715 Bings Birthday/P7140011 - Ferney-...   
91543  /2019/190801 Schweizer Nationalfeiertag/P80100...   
91544  /2019/190801 Schweizer Nationalfeiertag/P80100...   
91545  /2019/190801 Schweizer Nationalfeiertag/P80100...   

                             Date  FocalLength in mm  ExposureTime  \
0      Jul 10 Mon ,2006, 11:36:06                6.3      0.001250   
1      Jul 10 Mon ,2006, 15:52:49                6.3      0.003125   
2      Jul 10 Mon ,2006, 16:17:26      